# Colab Server (FastAPI)
Khởi chạy server FastAPI trên Google Colab, hỗ trợ CORS, GET/POST, logging, và kiểm thử API.

In [ ]:
import sys, subprocess
subprocess.run([sys.executable, '-m', 'pip', 'install', '--quiet', 'fastapi', 'uvicorn', 'nest_asyncio', 'httpx'])


In [ ]:
API_BASE = 'https://miyoko-trichomonadal-reconditely.ngrok-free.dev'
INTERNAL_BASE = 'http://127.0.0.1:8000'
PORT = 8000


In [ ]:
import logging
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import time

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('colab_server')

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

class EchoBody(BaseModel):
    message: str
    meta: dict | None = None

@app.middleware('http')
async def log_requests(request: Request, call_next):
    logger.info(f'{request.method} {request.url}')
    start = time.time()
    response = await call_next(request)
    duration = round((time.time() - start) * 1000)
    logger.info(f'{response.status_code} in {duration}ms')
    return response

@app.get('/health')
async def health():
    return {'ok': True, 'ts': time.time()}

@app.get('/info')
async def info():
    return {'api_base': API_BASE, 'internal': INTERNAL_BASE, 'port': PORT}

@app.post('/echo')
async def echo(body: EchoBody):
    return {'received': body.dict(), 'ts': time.time()}


In [ ]:
import nest_asyncio
nest_asyncio.apply()
import threading
import uvicorn

def run_server():
    uvicorn.run(app, host='0.0.0.0', port=PORT, log_level='info')

thread = threading.Thread(target=run_server, daemon=True)
thread.start()
print(f'Server started at {INTERNAL_BASE}')


## Hướng dẫn sử dụng
- Mở cell cài đặt và chạy để cài dependencies.
- Chạy cell cấu hình để set `API_BASE`.
- Chạy cell khởi chạy server.
- Khi dùng Ngrok, cập nhật lại `API_BASE` đúng domain và dùng các cell test.
- Để tránh timeout Colab, có thể bật keep-alive ping ở cell dưới.

In [ ]:
import asyncio, httpx
keepalive_task = None

async def keepalive(interval_sec: int = 60):
    async with httpx.AsyncClient(timeout=10.0) as client:
        while True:
            try:
                await client.get(f'{API_BASE}/health')
            except Exception:
                pass
            await asyncio.sleep(interval_sec)

def start_keepalive(interval_sec: int = 60):
    global keepalive_task
    if keepalive_task is None:
        keepalive_task = asyncio.create_task(keepalive(interval_sec))
    print('Keep-alive started')


## Kiểm thử nội bộ
Gọi API qua địa chỉ nội bộ.

In [ ]:
import httpx
r1 = httpx.get(f'{INTERNAL_BASE}/health', timeout=10.0)
r2 = httpx.post(f'{INTERNAL_BASE}/echo', json={'message':'hello','meta':{'source':'colab'}}, timeout=10.0)
print(r1.json())
print(r2.json())


## Kiểm thử qua API_BASE
Gọi API qua domain Ngrok.

In [ ]:
import httpx
r1 = httpx.get(f'{API_BASE}/health', timeout=10.0)
r2 = httpx.post(f'{API_BASE}/echo', json={'message':'hello','meta':{'source':'client'}}, timeout=10.0)
print(r1.json())
print(r2.json())
